In [22]:
import asyncio

from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.functions.kernel_arguments import KernelArguments

from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
    AzureChatPromptExecutionSettings,
)
import os
from dotenv import load_dotenv
load_dotenv()


endpoint = os.environ.get('AZURE_OPENAI_RESOURCE')
model_name = os.environ.get('AZURE_OPENAI_MODEL')
deployment = os.environ.get('AZURE_OPENAI_MODEL')
subscription_key = os.environ.get('AZURE_OPENAI_KEY')

# Initialize the kernel
kernel = Kernel()

# Add Azure OpenAI chat completion
chat_completion = AzureChatCompletion(
    deployment_name=deployment,
    api_key=subscription_key,
    endpoint=endpoint,
)
kernel.add_service(chat_completion)


In [23]:
import logging

# Get root logger
root_logger = logging.getLogger("kernel")
root_logger.setLevel(logging.DEBUG)

# Remove all existing handlers
for handler in root_logger.handlers[:]:
    root_logger.removeHandler(handler)

# Create file handler to overwrite the log file
f_handler = logging.FileHandler('my_log.log', mode='w')
f_handler.setLevel(logging.DEBUG)

# Formatter
formatter = logging.Formatter('[%(asctime)s - %(name)s:%(lineno)d - %(levelname)s] %(message)s',
                              datefmt='%Y-%m-%d %H:%M:%S')
f_handler.setFormatter(formatter)

# Add file handler only to root logger
root_logger.addHandler(f_handler)

# Optionally adjust specific logger levels, e.g., semantic kernel logger
logging.getLogger("kernel").setLevel(logging.DEBUG)
logging.getLogger("kernel").propagate = True  # allow logs to bubble to root

In [24]:
chat_completion : AzureChatCompletion = kernel.get_service(type=ChatCompletionClientBase)

Sample plugin

In [25]:
from typing import Annotated
from semantic_kernel.functions import kernel_function

class LightsPlugin:
    lights = [
        {"id": 1, "name": "Table Lamp", "is_on": False},
        {"id": 2, "name": "Porch light", "is_on": False},
        {"id": 3, "name": "Chandelier", "is_on": True},
    ]

    @kernel_function(
        name="get_lights",
        description="Gets a list of lights and their current state",
    )
    def get_state(
        self,
    ) -> str:
        """Gets a list of lights and their current state."""
        return self.lights

    @kernel_function(
        name="change_state",
        description="Changes the state of the light",
    )
    def change_state(
        self,
        id: int,
        is_on: bool,
    ) -> str:
        """Changes the state of the light."""
        for light in self.lights:
            if light["id"] == id:
                light["is_on"] = is_on
                return light
        return None

In [26]:
# Add the plugin to the kernel
kernel.add_plugin(
    LightsPlugin(),
    plugin_name="Lights",
)

KernelPlugin(name='Lights', description=None, functions={'change_state': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='change_state', plugin_name='Lights', description='Changes the state of the light', parameters=[KernelParameterMetadata(name='id', description=None, default_value=None, type_='int', is_required=True, type_object=<class 'int'>, schema_data={'type': 'integer'}, include_in_function_choices=True), KernelParameterMetadata(name='is_on', description=None, default_value=None, type_='bool', is_required=True, type_object=<class 'bool'>, schema_data={'type': 'boolean'}, include_in_function_choices=True)], is_prompt=False, is_asynchronous=False, return_parameter=KernelParameterMetadata(name='return', description='', default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string'}, include_in_function_choices=True), additional_properties={}), invocation_duration_histogram=<opentelemetry.metrics._internal.instrument._ProxyH

Auto agentic behavior

In [27]:
execution_settings = AzureChatPromptExecutionSettings()
execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

In [28]:
# Initiate a back-and-forth chat
userInput = None

# Create a history of the conversation
history = ChatHistory()

exit_commands = {"exit", "quit", "bye"}
while True:
    # Collect user input
    userInput = input("User > ").strip()

    if userInput.lower() in exit_commands:
        print("Exiting chat. Goodbye!")
        break

    # Add user input to the history
    history.add_user_message(userInput)

    # Get the response from the AI
    result = await chat_completion.get_chat_message_content(
        chat_history=history,
        settings=execution_settings,
        kernel=kernel,
    )

    # Print the results
    print("Assistant > " + str(result))

    # Add the message from the agent to the chat history
    history.add_message(result)

Assistant > Hello! How can I help you today?
Exiting chat. Goodbye!
